In [2]:
"""
Practice task: Cohort retention analysis (learning exercise)

I reproduced this code as part of my Python practice.
Goal: understand cohort logic and be able to explain and modify the solution.

Business questions:
- How many users return in month 2, 3, 4 after first activity?
- Which cohorts retain better over time?
- Where do we see the biggest drop-off?
"""

import pandas as pd
import numpy as np

np.random.seed(42)

users = range(1, 101)
start_dates = pd.date_range("2024-01-01", periods=6, freq="MS")

data = []

for user in users:
    cohort_start = np.random.choice(start_dates)
    active_months = np.random.randint(1, 6)

    for i in range(active_months):
        event_date = cohort_start + pd.DateOffset(months=i) + pd.DateOffset(days=np.random.randint(0, 10))
        data.append({
            "user_id": user,
            "event_date": event_date
        })

df = pd.DataFrame(data).sort_values(["user_id", "event_date"]).reset_index(drop=True)
df.head()
df["event_date"] = pd.to_datetime(df["event_date"])
df["event_month"] = df["event_date"].dt.to_period("M")
df[["user_id", "event_date", "event_month"]].head()
first_month = df.groupby("user_id")["event_month"].min()
df = df.merge(first_month.rename("cohort_month"), on="user_id")
df.head()
def cohort_period(event_month, cohort_month):
  return (event_month.year - cohort_month.year) * 12 + (event_month.month - cohort_month.month) + 1
df["cohort_period"] = df.apply(lambda row: cohort_period(row["event_month"], row["cohort_month"]),
axis=1)
df[["user_id", "cohort_month", "event_month", "cohort_period"]].head(10)
cohort_counts = (df.groupby(["cohort_month", "cohort_period"])["user_id"]
                 .nunique()
                 .reset_index())
cohort_counts.head()
cohort_table = cohort_counts.pivot_table(
    index="cohort_month",
    columns="cohort_period",
    values="user_id")
cohort_table
cohort_size = cohort_table.iloc[:, 0]
retention_table = cohort_table.divide(cohort_size, axis=0).round(3)
retention_table
active_months = df.groupby("user_id")["event_month"].nunique()
valid_users = active_months[active_months >= 2].index
df2 = df[df["user_id"].isin(valid_users)]
df2

,user_id,event_date,event_month,cohort_month,cohort_period
0,1,2024-04-08,2024-04,2024-04,1
1,1,2024-05-05,2024-05,2024-04,2
2,1,2024-06-07,2024-06,2024-04,3
3,1,2024-07-10,2024-07,2024-04,4
4,1,2024-08-03,2024-08,2024-04,5
...,...,...,...,...,...
311,99,2024-06-01,2024-06,2024-06,1
312,99,2024-07-07,2024-07,2024-06,2
313,99,2024-08-07,2024-08,2024-06,3
314,99,2024-09-09,2024-09,2024-06,4
